In [177]:
import pandas as pd
import yfinance as yf
from binance.client import Client
client = Client()

#dfSPY = pd.read_csv("SPY.USUSD_Candlestick_1_D_BID_16.02.2017-21.05.2022.csv")
#dfSPY=yf.download("^GSPC",start='2011-01-05', end='2021-01-05')
#dfSPY=yf.download("^RUI",start='2011-01-05', end='2021-01-05')
#dfSPY=yf.download("EURUSD=X",start='2011-01-05', end='2021-01-05')

# dfSPY=dfSPY[dfSPY.High!=dfSPY.Low]
# dfSPY.reset_index(inplace=True)
# dfSPY.head()

def getdata(symbol,start):
    frame = pd.DataFrame(client.get_historical_klines(symbol,'4h',start))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time','Open','High','Low','Close','Volume']
    frame.set_index('Time',inplace=True)
    frame.index = pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    return frame

dfSPY = getdata('ETHUSDT','2019-01-01')
dfSPY.head()


,Open,High,Low,Close,Volume
Time,,,,,
2019-01-01 00:00:00,131.45,132.22,130.00,131.32,40579.15620
2019-01-01 04:00:00,131.31,134.30,131.30,133.56,52372.53471
2019-01-01 08:00:00,133.56,135.30,133.19,133.75,44277.02066
2019-01-01 12:00:00,133.78,136.00,132.26,132.98,53121.18167
2019-01-01 16:00:00,132.97,135.58,131.00,135.27,51162.59672


In [178]:
import pandas_ta as ta
dfSPY['EMA']=ta.sma(dfSPY.Close, length=50)# adds SMA can be changed to whatever
dfSPY['RSI']=ta.rsi(dfSPY.Close,length=2) # adds RSI
#dfSPY.ta.indicators()
#help(ta.bbands)
my_bbands = ta.bbands(dfSPY.Close, length=20, std=1.5)
my_bbands[0:50]
#my_bbands
dfSPY=dfSPY.join(my_bbands)
dfSPY.dropna(inplace=True)
dfSPY.reset_index(inplace=True)
dfSPY

,Time,Open,High,Low,Close,Volume,EMA,RSI,BBL_20_1.5,BBM_20_1.5,BBU_20_1.5,BBB_20_1.5,BBP_20_1.5
0,2019-01-09 04:00:00,150.30,150.90,149.08,149.80,42130.82725,148.3108,63.607705,147.141865,150.4870,153.832135,4.445746,0.397314
1,2019-01-09 08:00:00,149.80,150.58,149.03,149.41,34146.17067,148.6726,47.386234,147.067965,150.4315,153.795035,4.471849,0.348151
2,2019-01-09 12:00:00,149.45,149.78,147.41,148.26,60621.24267,148.9666,18.924322,146.911011,150.3465,153.781989,4.570095,0.196331
3,2019-01-09 16:00:00,148.26,149.25,147.97,148.58,27914.86464,149.2632,39.235809,146.752636,150.1505,153.548364,4.525944,0.268899
4,2019-01-09 20:00:00,148.56,149.90,147.07,148.10,57626.98516,149.5656,22.400301,146.559240,150.0105,153.461760,4.601358,0.223217
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8615,2022-12-15 20:00:00,1275.03,1275.49,1259.27,1266.54,52052.64190,1276.0974,16.518651,1259.295115,1295.9205,1332.545885,5.652412,0.098905
8616,2022-12-16 00:00:00,1266.53,1273.87,1265.65,1271.79,30822.46400,1276.8910,47.623077,1261.952955,1296.7810,1331.609045,5.371461,0.141223
8617,2022-12-16 04:00:00,1271.78,1280.06,1267.13,1277.77,31886.49810,1277.8228,71.669750,1262.226865,1296.8995,1331.572135,5.347004,0.224141
8618,2022-12-16 08:00:00,1277.77,1279.27,1205.50,1212.83,223337.36950,1277.4422,6.532416,1250.519895,1294.0585,1337.597105,6.729001,-0.432833


In [179]:
def addemasignal(df, backcandles):
    emasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.EMA[i]:
                dnt=0
            if df.Low[i]<=df.EMA[i]:
                upt=0
        if upt==1 and dnt==1:
            #print("!!!!! check trend loop !!!!")
            emasignal[row]=3
        elif upt==1:
            emasignal[row]=2
        elif dnt==1:
            emasignal[row]=1
    df['EMASignal'] = emasignal

addemasignal(dfSPY, 6)

In [180]:
def addorderslimit(df, percent):
    ordersignal=[0]*len(df)
    BBL = 'BBL_20_1.5'
    BBU = 'BBU_20_1.5'
    for i in range(1, len(df)): #EMASignal of previous candle!!! modified!!!
        if df.EMASignal[i]==2 and df.Close[i]<=df[BBL][i]:# and df.RSI[i]<=100: #Added RSI condition to avoid direct close condition
            ordersignal[i]=df.Close[i]-df.Close[i]*percent
        elif df.EMASignal[i]==1 and df.Close[i]>=df[BBU][i]:# and df.RSI[i]>=0:
            ordersignal[i]=df.Close[i]+df.Close[i]*percent
    df['ordersignal']=ordersignal
    
addorderslimit(dfSPY, 0.00)

In [181]:
dfSPY[dfSPY.ordersignal!=0]

,Time,Open,High,Low,Close,Volume,EMA,RSI,BBL_20_1.5,BBM_20_1.5,BBU_20_1.5,BBB_20_1.5,BBP_20_1.5,EMASignal,ordersignal
132,2019-01-31 04:00:00,108.99,109.39,108.40,109.14,29728.02599,111.7262,89.747612,102.757896,105.7810,108.804104,5.715780,1.055555,1,109.14
494,2019-04-01 16:00:00,141.65,141.99,140.42,140.49,31188.60709,138.5830,5.786564,140.575005,141.7450,142.914995,1.650845,-0.036327,2,140.49
573,2019-04-14 20:00:00,163.45,168.11,163.43,167.23,58677.96976,169.3398,98.879871,161.516921,163.4205,165.324079,2.329670,1.500615,1,167.23
574,2019-04-15 00:00:00,167.21,168.81,166.67,167.02,34926.96134,169.2902,89.858558,161.334266,163.5245,165.714734,2.678783,1.297974,1,167.02
575,2019-04-15 04:00:00,166.99,168.50,166.09,166.73,35879.20919,169.3148,71.773009,161.232668,163.6575,166.082332,2.963301,1.133549,1,166.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7935,2022-08-24 12:00:00,1650.91,1688.00,1642.44,1683.50,168398.54540,1706.9458,88.467598,1570.454068,1617.8065,1665.158932,5.853906,1.193666,1,1683.50
7936,2022-08-24 16:00:00,1683.49,1694.13,1664.00,1678.59,112378.98260,1702.3848,75.241769,1569.078093,1620.3985,1671.718907,6.334295,1.066943,1,1678.59
7967,2022-08-29 20:00:00,1531.36,1559.99,1529.21,1551.80,96889.58590,1581.3298,96.137130,1444.518414,1493.0275,1541.536586,6.498083,1.105789,1,1551.80
7968,2022-08-30 00:00:00,1551.79,1552.66,1523.56,1544.20,92145.75710,1579.7302,72.308413,1445.933567,1492.2485,1538.563433,6.207402,1.060850,1,1544.20


# Visualization

In [182]:
import numpy as np
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
dfSPY['pointposbreak'] = dfSPY.apply(lambda row: pointposbreak(row), axis=1)

In [185]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = dfSPY[1000:1250].copy()
#dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
#dfpl.reset_index(inplace=True)
BBL = 'BBL_20_1.5'
BBU = 'BBU_20_1.5'
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl[BBL], line=dict(color='blue', width=1), name=BBL),
                go.Scatter(x=dfpl.index, y=dfpl[BBU], line=dict(color='blue', width=1), name=BBU)])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [ ]:
dfpl = dfSPY[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [ ]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.05
    ordertime=[]
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        
        for j in range(0, len(self.orders)):
            #print('!!!!!!!!!!!!!!!!!!!', self.data.index[-1])
            if self.data.index[-1]-self.ordertime[0]>5:#days max to fulfill the order!!!
                #print('----------------------')
                #print(self.orders)
                #print(self.ordertime)
                self.orders[0].cancel()
                self.ordertime.pop(0)   
            
        if len(self.trades)>0:
            #print(self.data.index[-1], self.trades)
            if self.data.index[-1]-self.trades[-1].entry_time>=10:
                self.trades[-1].close()
                #print(self.data.index[-1], self.trades[-1].entry_time)
            
            if self.trades[-1].is_long and self.data.RSI[-1]>=50:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=50:
                self.trades[-1].close()
        
        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.buy(sl=self.signal/2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.sell(sl=self.signal*2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/10, commission=.00)
stat = bt.run()
stat

/var/folders/_s/_4ktyf8904n71s9jshmj85500000gn/T/ipykernel_2159/3947618467.py:52: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    8619.0
Duration                               8619.0
Exposure Time [%]                    0.765661
Equity Final [$]                      9540.17
Equity Peak [$]                      10061.77
Return [%]                            -4.5983
Buy & Hold Return [%]              708.938585
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -7.166731
Avg. Drawdown [%]                   -7.166731
Max. Drawdown Duration                 8043.0
Avg. Drawdown Duration                 8043.0
# Trades                                 10.0
Win Rate [%]                             30.0
Best Trade [%]                       1.935954
Worst Trade [%]                     -4.802481
Avg. Trade [%]                    

In [ ]:
bt.plot(show_legend=False)

Row(id='15558', ...)

In [ ]:
stat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-29,574,577,167.23,165.14,60.61,0.012498,574,577,3
1,-25,1325,1332,193.92,198.04,-103.00,-0.021246,1325,1332,7
2,30,2737,2741,163.37,158.19,-155.40,-0.031707,2737,2741,4
3,-24,2938,2947,198.15,199.42,-30.48,-0.006409,2938,2947,9
4,-13,3655,3659,370.36,363.19,93.21,0.019360,3655,3659,4
5,-1,6793,6801,3025.12,3135.64,-110.52,-0.036534,6793,6801,8
6,1,7061,7064,3283.54,3279.41,-4.13,-0.001258,7061,7064,3
7,-2,7417,7424,1880.07,1970.36,-180.58,-0.048025,7417,7424,7
8,-4,7629,7635,1101.00,1112.21,-44.84,-0.010182,7629,7635,6
9,2,7799,7804,1622.27,1629.92,15.30,0.004716,7799,7804,5
